In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from GA_functions import fitness, select_parents, crossover, mutation, GA_algorithm, conc_to_spectra, normalize_and_pca

## Many iterations

In [2]:
#This block of code creates random concentrations and spectra to start the algorithm. 
Sample_concentrations = pd.read_excel('Sample_concentrations.xlsx')
red_conc = Sample_concentrations[['Red vol frac']].values
green_conc = Sample_concentrations[['Green vol frac']].values
blue_conc = Sample_concentrations[['Blue vol frac']].values
sample_conc = np.hstack((red_conc, green_conc, blue_conc))

sample_spectra = pd.read_excel('Sample_spectra.xlsx')
sample_spectra = np.asarray(sample_spectra)
red = sample_spectra[:,1]
green = sample_spectra[:,2]
blue = sample_spectra[:,3]
wavelength = sample_spectra[:,0]

conc_array = np.random.dirichlet((1, 1, 1), 10)
spectra_array = conc_to_spectra(conc_array, red, green, blue, wavelength)

In [3]:
#best_conc_lst = []
#for i in range(20):


y_train = sample_conc[0:-1,:]
x_train = sample_spectra[:,1:-1]
objective_row = -1 #Change this number for a different concentration to test. 
x_test = sample_spectra[:,objective_row].reshape(-1,1) 
print('The target conc is:', sample_conc[objective_row,:])
print(conc_array)
print()

######## Use this if the starting concentrations and spectra from excel are to be used ###########
#y_train = sample_conc[0:-1,:]
#y_train_conc = y_train
##################################################################################################


######## Use this for random starting concentrations and spectra #################################
x_train = spectra_array.T
y_train_conc = conc_array 
##################################################################################################


#Normalizae x_train
x_train = MinMaxScaler().fit(x_train).transform(x_train)
x_test = MinMaxScaler().fit(x_test).transform(x_test).T
x_train = x_train.T
x_test = x_test.reshape(1,-1)[0].reshape(-1,1).T

pca = PCA(n_components=3)
x_train_new = pca.fit(x_train)
x_train_spectra = pca.transform(x_train)
x_test = pca.transform(x_test)

best_gen0 = fitness(x_train_spectra, y_train_conc, x_test)
best_conc = best_gen0[-1,x_train_spectra.shape[1]:-1]

gen_concs = [y_train_conc]
best_conc_list = [best_conc]
for i in range(40):
    gen2_spectra = conc_to_spectra(gen_concs[i], red, green, blue, wavelength)
    #gen2_spectra = normalize_and_pca(gen2_spectra)
    x_train = gen2_spectra.T
    x_train = MinMaxScaler().fit(x_train).transform(x_train)
    x_train = x_train.T

    gen2_spectra = pca.transform(x_train)
     
    gen3 = GA_algorithm(gen2_spectra, gen_concs[i], x_test, 20, 10, 0.8)
    gen_concs.append(gen3)
    print(gen3)
    print()
    #best_gen5 = fitness(gen2_spectra, gen_concs[i], x_test)
    #best_conc = best_gen5[-1,3:-1]
    #best_conc_list.append(best_conc)


The target conc is: [0.5073313  0.01445543 0.47821328]
[[2.91682873e-01 1.24330213e-03 7.07073825e-01]
 [7.87538743e-01 9.93739223e-02 1.13087334e-01]
 [3.09004064e-02 8.13477962e-04 9.68286116e-01]
 [4.15538454e-01 5.55114716e-02 5.28950074e-01]
 [6.16543171e-01 1.56250995e-01 2.27205834e-01]
 [7.53234769e-02 6.69189311e-01 2.55487212e-01]
 [7.52623053e-01 1.46820952e-01 1.00555995e-01]
 [6.15761294e-01 1.48602319e-01 2.35636388e-01]
 [4.10183490e-01 3.51912655e-01 2.37903856e-01]
 [1.61558961e-01 5.01927767e-02 7.88248263e-01]]

[[0.33988534 0.44881245 0.21130221]
 [0.54248862 0.04172989 0.41578149]
 [0.66031746 0.13650794 0.2031746 ]
 [0.38447088 0.09267376 0.52285535]
 [0.24782135 0.46568627 0.28649237]
 [0.23822128 0.45050291 0.31127581]
 [0.39379699 0.05357143 0.55263158]
 [0.40277778 0.05277778 0.54444444]
 [0.29499404 0.27115614 0.43384982]
 [0.42492339 0.33299285 0.24208376]]

[[0.68209877 0.1404321  0.17746914]
 [0.53916211 0.03734062 0.42349727]
 [0.48506151 0.06239016 0.452

[[0.52169811 0.03018868 0.44811321]
 [0.39195231 0.03129657 0.57675112]
 [0.52208835 0.00200803 0.47590361]
 [0.37606838 0.35754986 0.26638177]
 [0.28950276 0.44309392 0.26740331]
 [0.19578578 0.03687445 0.76733977]
 [0.39925834 0.01483313 0.58590853]
 [0.51565558 0.02152642 0.462818  ]
 [0.5420082  0.00307377 0.45491803]
 [0.37926033 0.29151559 0.32922408]]

[[0.3692705  0.19496449 0.43576501]
 [0.31714286 0.00428571 0.67857143]
 [0.39505247 0.02398801 0.58095952]
 [0.50336215 0.03073967 0.46589817]
 [0.47593097 0.27429609 0.24977293]
 [0.41534989 0.00150489 0.58314522]
 [0.49382716 0.05887939 0.44729345]
 [0.42825607 0.22222222 0.34952171]
 [0.3378703  0.04163331 0.6204964 ]
 [0.28151709 0.48076923 0.23771368]]

[[0.50378072 0.06616257 0.43005671]
 [0.52161383 0.03554275 0.44284342]
 [0.49261084 0.04926108 0.45812808]
 [0.55455801 0.15883978 0.28660221]
 [0.28373894 0.29314159 0.42311947]
 [0.40552651 0.24645258 0.34802091]
 [0.64015152 0.02525253 0.33459596]
 [0.47363465 0.08474576 

In [4]:
mean_red = []
mean_green = []
mean_blue = []

for i in range(len(gen_concs)):
    mean_row = np.mean(gen_concs[i], axis = 0)
    mean_red.append(mean_row[0])
    mean_green.append(mean_row[1])
    mean_blue.append(mean_row[2])
print(np.mean(mean_red))
print(np.mean(mean_green))
print(np.mean(mean_blue))

    

0.4522013176605565
0.10370440929983761
0.4440942730396058


In [5]:
np.mean(gen3, axis=0)

array([0.39808697, 0.1427348 , 0.45917822])

In [6]:
x_test

array([[-0.53074881,  0.41636019, -0.51890462]])